# Analytical vs FEM Solution for Terzaghi 1D Consolidation

This notebook compares the analytical solution of Terzaghi’s one-dimensional consolidation equation with a numerical solution obtained using the finite element method (FEM) via FEniCSx.  
The objective is to quantify the error between the two approaches when predicting consolidation settlement.


## Problem definition

A single-layer soil profile is considered. The same parameters are used for both the analytical and FEM solutions. All model parameters are define below and are printed for reference.


## Model parameters

The parameters governing the analytical and FEM solutions are displayed below.

## Analytical solution

- The analytical solution is evaluated using a truncated Fourier series, with the number of terms defined in the code (`N_terms`).  
- A single-drainage condition is assumed.  
- The formulation follows standard expressions presented in *Craig’s Soil Mechanics*.  
- The initial condition assumes a uniform excess pore pressure distribution with depth due to instantaneous loading. This approach has been used as the analytical solution for this is given.


## Assumptions and scope

- Terzaghi’s consolidation theory is a one-dimensional formulation and considers consolidation only in the vertical (depth) direction.  
- The initial loading is represented as a uniform excess pore pressure distribution.  
- Stress distributions based on 2:1 dispersion or Boussinesq theory are not considered here and are examined separately in another notebook.



## Purpose of comparison

The analytical solution is used as a reference solution to evaluate the accuracy of the FEM based 1D Terzaghi consolidation model implemented in FEniCSx.  
Error measures, including RMSE and normalised \(L^2\) norms, are evaluated over time, with particular attention paid to early-time behaviour where discretisation effects are most pronounced.

# Limitations
- The accuracy of both the analytical and FEM solutions is sensitive to the choice of spatial and discretisation parameters.
- The use of coarse meshes can lead to inaccurate representation of steep pore pressure gradients that develop near the drained boundary at early times. Hence a large error occuring at this stage.
- Early time boundary layers cannot be adequately resolved by using coarse spatial discretisation,. These can result in early settlement and inflated error metrics (e.g. RMSE and normalised \(L^2\) norms).
- Large time steps can further implicate early discrepancies.


In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from matplotlib.animation import FuncAnimation 
from IPython.display import HTML
import matplotlib.dates as mdates


# set up to allow juypter notebook to use local modulse from src 
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../scripts'))
sys.path.insert(0, module_path)

from terazaghi_1d.fea_fenicsx import Get_Terazaghi1D_FEA 
from terazaghi_1d.analytical import Get_Terazaghi1d_Analytical


# parameters 
H = 5
num = 200
nodes = num + 1
P = 100 # stress applied 
Tx = 60*60*24*365 # seconds to a year Please keeps this within days 
time_step = 10000
dt = Tx / time_step
Cv = 2e-7 # m^2/s (coefficient of consolidation)
Mv = 5e-4 # 1/kPa  (or m^2/kN)

time_factor = (Cv * dt) / H**2

# n terms for analitical solution 
N_terms = 1000

total_settlement = Mv*P*H

print("\nModel parameters used for analytical and FEM solutions:")
print("--------------------------------------------------------")
print(f"Layer thickness,             = {H:.2f} m")
print(f"Applied surface pressure,    = {P:.2f} kPa")
print(f"Simulation duration,         = {Tx/(60*60*24):.2f} days")
print(f"Number of time steps         = {time_step}")
print(f"Time step size               = {(dt/(60*60*24)):.2f} days")
print(f"Coefficient of consolidation = {Cv:.2e} m^2/s")
print(f"Coefficient of volume comp.  = {Mv:.2e} m^2/kN\n")
print(f"Number of elements (FEM)     = {num}")
print(f"Number of nodes (FEM)        = {nodes}\n")
print(f"Analytical series terms      = {N_terms}\n")
print(f"Dimensionless time factor    = {time_factor:.4f}")
print(f"Total Settlement             = {total_settlement:.4f} m")



Model parameters used for analytical and FEM solutions:
--------------------------------------------------------
Layer thickness,             = 5.00 m
Applied surface pressure,    = 100.00 kPa
Simulation duration,         = 365.00 days
Number of time steps         = 10000
Time step size               = 0.04 days
Coefficient of consolidation = 2.00e-07 m^2/s
Coefficient of volume comp.  = 5.00e-04 m^2/kN

Number of elements (FEM)     = 200
Number of nodes (FEM)        = 201

Analytical series terms      = 1000

Dimensionless time factor    = 0.0000
Total Settlement             = 0.2500 m


# FEniCSx Solver

In [2]:
# plotting fenicsx data. to note purposely choicen different t or time_step+1 as fenicsx gives t+1 results due to uh ands 
fem_cdata = Get_Terazaghi1D_FEA(H, num, P, Tx, time_step, Cv, 0, True) # 0 as we are using uniform force 
Z = -np.linspace(0, H, num = nodes)
T = np.linspace(0,(Tx/(60*60*24)), num= time_step)
fem_cdata = pd.DataFrame(fem_cdata, columns = Z, index = T)

fem_cdata

,-0.000,-0.025,-0.050,-0.075,-0.100,-0.125,-0.150,-0.175,-0.200,-0.225,...,-4.775,-4.800,-4.825,-4.850,-4.875,-4.900,-4.925,-4.950,-4.975,-5.000
0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
0.036504,1.0,0.422661,0.149142,0.052627,0.018570,0.006553,0.002312,0.000816,0.000288,0.000102,...,4.440892e-16,3.330669e-16,3.330669e-16,3.330669e-16,3.330669e-16,5.551115e-16,4.440892e-16,4.440892e-16,3.330669e-16,3.330669e-16
0.073007,1.0,0.581227,0.290546,0.132676,0.057457,0.024029,0.009804,0.003927,0.001551,0.000605,...,8.881784e-16,6.661338e-16,8.881784e-16,6.661338e-16,6.661338e-16,8.881784e-16,6.661338e-16,6.661338e-16,5.551115e-16,4.440892e-16
0.109511,1.0,0.660448,0.389581,0.209980,0.105777,0.050656,0.023338,0.010431,0.004550,0.001946,...,1.443290e-15,9.992007e-16,1.332268e-15,1.110223e-15,1.110223e-15,1.332268e-15,1.110223e-15,9.992007e-16,6.661338e-16,5.551115e-16
0.146015,1.0,0.708192,0.459461,0.275612,0.154742,0.082276,0.041839,0.020509,0.009752,0.004519,...,1.887379e-15,1.554312e-15,1.554312e-15,1.443290e-15,1.554312e-15,1.554312e-15,1.443290e-15,1.443290e-15,1.110223e-15,9.992007e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364.853985,1.0,0.994596,0.989192,0.983788,0.978386,0.972985,0.967586,0.962189,0.956794,0.951403,...,3.198398e-01,3.194891e-01,3.191797e-01,3.189114e-01,3.186844e-01,3.184987e-01,3.183542e-01,3.182510e-01,3.181891e-01,3.181684e-01
364.890489,1.0,0.994596,0.989192,0.983789,0.978387,0.972987,0.967588,0.962191,0.956797,0.951406,...,3.198814e-01,3.195307e-01,3.192213e-01,3.189530e-01,3.187261e-01,3.185403e-01,3.183959e-01,3.182927e-01,3.182307e-01,3.182101e-01
364.926993,1.0,0.994596,0.989193,0.983790,0.978389,0.972989,0.967590,0.962194,0.956800,0.951409,...,3.199230e-01,3.195723e-01,3.192629e-01,3.189947e-01,3.187677e-01,3.185820e-01,3.184375e-01,3.183343e-01,3.182724e-01,3.182518e-01
364.963496,1.0,0.994597,0.989194,0.983791,0.978390,0.972990,0.967592,0.962196,0.956803,0.951412,...,3.199646e-01,3.196139e-01,3.193045e-01,3.190363e-01,3.188093e-01,3.186236e-01,3.184792e-01,3.183760e-01,3.183141e-01,3.182934e-01


# Analytical Solver

In [3]:
# analytical solution plotitng 
analytical_cdata, a_Z, = Get_Terazaghi1d_Analytical(H, Tx, time_step, num, Cv, N_terms)
a_T = np.linspace(0, Tx/(60*60*24), time_step, dtype=float)
analytical_cdata = pd.DataFrame(analytical_cdata, columns= -a_Z, index= a_T)

analytical_cdata

KeyboardInterrupt: 

# Computing for Error 

In [ ]:
# assumes that the t and z for both data is identical in which it is 
error = fem_cdata - analytical_cdata
#RMSE error per time step 
RMSE = np.sqrt((error**2).mean(axis = 1))
# Normalised L2 norm error 
num = (error**2).sum(axis = 1)
den = (analytical_cdata**2).sum(axis = 1)
E_L2 = np.sqrt(num / (den+ 1e-12))
time = np.linspace(0, (Tx/(60*60*24)), time_step)


# plotting err
plt.figure()
plt.plot(time, RMSE, label = "RMSE Error")
plt.plot([0,np.max(time)],[0,0], linestyle ="dotted")  # Please change for capture of map
plt.plot(time, E_L2, label = "Normalised L2 norm Error")
plt.xlabel("Time (days)")
plt.legend()
plt.title("Error between FEM and Analytical Solution")
plt.show()


In [ ]:

# drawing settlement with time plotting 
analytical_settlement= analytical_cdata.mean(axis=1)*total_settlement
fem_settlement = fem_cdata.mean(axis=1)*total_settlement

plt.figure()
plt.plot(time, -analytical_settlement, label="Analytical Settlement")
plt.plot(time, -fem_settlement, label="FEM Settlement")
plt.xlabel("Time (days)")
plt.ylabel("Settlement in m")
plt.legend()
plt.title("settlement over time")
plt.show()
print(f"total settlement: {total_settlement} m")

print(f"total consolidation settlement reach. \nanalytical: {np.max(analytical_settlement):.4f}m \nFEM: {np.max(fem_settlement):.4}m \nactual total settlememt: {total_settlement:.4} ")

# Heat Map Representation through out time 
note: these use local degree of consolidation and not settlement

In [ ]:
time = time
depth = Z

kx = max(1, len(time)//10)    # ~8 labels across, auto
ky = max(1, len(depth)//10)  # ~10 labels down, auto 


ax = sns.heatmap(abs(error).T, annot=False, cmap="YlGnBu", 
                 xticklabels=time, yticklabels=depth)

ax.set_xticks(np.arange(0, len(time), kx) + 0.5)
ax.set_xticklabels([f"{time[i]:.1f}" for i in range(0, len(time), kx)],
                   rotation=0)

ax.set_yticks(np.arange(0, len(depth), ky) + 0.5)
ax.set_yticklabels([f"{depth[i]:.1f}" for i in range(0, len(depth), ky)],
                   rotation=0)

ax.set_xlabel("Time (Days)")
ax.set_ylabel("Depth (m)")
ax.set_title("absolute error of Analytical solution and FEM")
plt.tight_layout()
plt.show()
